# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [14]:
%matplotlib inline
!pip install emoji
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [15]:
import os

filename = 'rtx 3080.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo rtx 3080.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [16]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,@rtx_3080 @thelockalarm don’t let em get frost...,I
1,answering your rtx 3080 questions! - probing p...,R
2,are you still doing my mom https://t.co/yn0umv...,I
3,i guess us customs wins this round. a partner ...,P
4,gonna order when listing on amazon us!\n \nhtt...,I


In [17]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,@kaylasweetx i just want a rtx 3080,P
1,plan on upgrading with @nvidia's rtx 3080? be ...,I
2,@insomniacgames i thought the playstation 5 co...,P
3,@asus_rog @asus_rog are you guys releasing the...,R
4,@vixlyt @eryxies @charlieintel its 2020. let's...,R


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...   
"Relevante -> qualque conteúdo que influência opiniões - seja ele crítico ou informátivo (até porque memso informativo sempre vai ter uma imparcialidade)\n",
    "\n",
    "Pouco Relevante -> passa alguma informação menor, menos importante pra formar opinião do produto, como por exemplo o fato de alguem vai comprar, ou críticas que nao sao culpa da nvidia(fabricante do produto)\n",
    "\n",
    "Irrelevante -> todo o resto, principalmente tweets que nem comentam o produto."

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [18]:
def separate_emoji(text): #separa emojis dos outros caracteres
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip() #caso seja um emoji, adiciona um espaco, caso nao seja, apenas retorna a letra.

In [19]:
chars = [",", "'", '"', ".", "!", "?", ";", ":", "[", "]","{", "}", "(", ")", "#", "/",  "\\", "https","+", "@", "*", "-"] 
word_count = dict()

train.Classificação = train.Classificação.apply(lambda x: x.upper())
for i, tweet in train.iterrows(): #iterando a base de dados
    piu = tweet.Treinamento #carregando apenas os tweets
    
    for c in chars: #iterando a listad e cacarteres p/ remover
        piu = piu.replace(c, " ") #trocando os caracteres por " "
        
    for word in separate_emoji(piu).split(): #splitando os tweets em palavras e emojis separadamente
        word_count[word] = word_count.get(word, {'I':0, 'P':0, 'R':0}) #garantindo que  a palavra existe no dicionario, caso nao exista ela eh criada
        word_count[word][tweet.Classificação]+=1 #incrementando o contador do valor correto
        
I = {word: word_count[word]['I'] for word in word_count if word_count[word]['I'] > 0} #I contem todas as palavras Irrelevantes, seu sum(I.values) == total de palavras Irrelevantes
P = {word: word_count[word]['P'] for word in word_count if word_count[word]['P'] > 0} #P contem todas as palavras Pouco relevantes, seu sum(P.values) == total de palavras Pouco relevantes
R = {word: word_count[word]['R'] for word in word_count if word_count[word]['R'] > 0} #R contem todas as palavras Relevantes, seu sum(R.values) == total de palavras Relevantes

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [20]:

total = len(word_count) #total de palavras no banco de dados
totalR = (sum(R.values())) #total de palavaras na categoria Relevante
totalP = (sum(P.values())) #total de palavaras na categoria Pouco Relevante
totalI = (sum(I.values())) #total de palavaras na categoria Irrelevante

l = [] #lista de classificacoes, mesma ordem da base test
for i, tweet in test.iterrows(): #iterando a base de dados
    
    #valores para a probabiliade do tweet ser Irrelevante, comeca como 1 para nao der erro na multiplicacao abaixo
    i = 1
    p = 1
    r = 1
    
    piu = tweet.Teste #carregando apenas os tweets 
    
    for c in chars: #iterando a listad e cacarteres p/ remover
        piu = piu.replace(c, " ") #trocando os caracteres por " "
    
    #splitando os tweets em palavras e emojis separadamente e calculando a probalidade de cada palavra de ser cada categoria, fazendo a suavizacao de laplace
    for word in separate_emoji(piu).split(): 
        
        #calculando a probabiliade de ser relevamte, pegamos a probabiliadde total e multiplicamos pelo valor da palavra
        r *= (R.get(word, 0) + 1) / (totalR + total) 
        p *= (P.get(word, 0) + 1) / (totalP + total) 
        i *= (I.get(word, 0) + 1) / (totalI + total) 
        
    result = {"R":r,"P":p,"I": i} #dicionario para a comparacao dentro do MAX abaixo
    l.append(max(result, key = result.get)) #compara os valores de r, p, i, e adiciona correta a lista L 

test["Classificação_A"] = l #adiciona a coluna classificacao a test, como a ordem eh a mesma, podemos adicionar.

test.head(20)

,Teste,Classificação,Classificação_A
0,@kaylasweetx i just want a rtx 3080,P,P
1,plan on upgrading with @nvidia's rtx 3080? be ...,I,R
2,@insomniacgames i thought the playstation 5 co...,P,R
3,@asus_rog @asus_rog are you guys releasing the...,R,P
4,@vixlyt @eryxies @charlieintel its 2020. let's...,R,R
5,@datbx_ i am not cool 😭,I,P
6,@dnpthree @newcuiture #freegamingpc cad to usd...,P,P
7,@colteastwood i’m getting the rtx 3080,P,R
8,@greencapt @chuckxl @bigmaki this all just see...,R,R
9,@flaw4q this going to be on my rtx 3080 i9 109...,P,P


In [21]:

total = len(word_count) #total de palavras no banco de dados
totalR = (sum(R.values())) #total de palavaras na categoria Relevante
totalP = (sum(P.values())) #total de palavaras na categoria Pouco Relevante
totalI = (sum(I.values())) #total de palavaras na categoria Irrelevante

l = [] #lista de classificacoes, mesma ordem da base test


for i, tweet in train.iterrows(): #iterando a base de dados
    
    #valores para a probabiliade do tweet ser Irrelevante, comeca como 1 para nao der erro na multiplicacao abaixo
    i = 1
    p = 1
    r = 1
    
    piu = tweet.Treinamento #carregando apenas os tweets 
    
    for c in chars: #iterando a listad e cacarteres p/ remover
        piu = piu.replace(c, " ") #trocando os caracteres por " "
    
    #splitando os tweets em palavras e emojis separadamente e calculando a probalidade de cada palavra de ser cada categoria, fazendo a suavizacao de laplace
    for word in separate_emoji(piu).split(): 
        
        #calculando a probabiliade de ser relevamte, pegamos a probabiliadde total e multiplicamos pelo valor da palavra
        r *= (R.get(word, 0) + 1) / (totalR + total) 
        p *= (P.get(word, 0) + 1) / (totalP + total) 
        i *= (I.get(word, 0) + 1) / (totalI + total) 
        
    result = {"R":r,"P":p,"I": i} #dicionario para a comparacao dentro do MAX abaixo
    l.append(max(result, key = result.get)) #compara os valores de r, p, i, e adiciona correta a lista L 

train["Classificação_A"] = l #adiciona a coluna classificacao a test, como a ordem eh a mesma, podemos adicionar.

test.head(20)

,Teste,Classificação,Classificação_A
0,@kaylasweetx i just want a rtx 3080,P,P
1,plan on upgrading with @nvidia's rtx 3080? be ...,I,R
2,@insomniacgames i thought the playstation 5 co...,P,R
3,@asus_rog @asus_rog are you guys releasing the...,R,P
4,@vixlyt @eryxies @charlieintel its 2020. let's...,R,R
5,@datbx_ i am not cool 😭,I,P
6,@dnpthree @newcuiture #freegamingpc cad to usd...,P,P
7,@colteastwood i’m getting the rtx 3080,P,R
8,@greencapt @chuckxl @bigmaki this all just see...,R,R
9,@flaw4q this going to be on my rtx 3080 i9 109...,P,P


In [22]:
print(test.Classificação.value_counts(normalize=True))
print(train.Classificação.apply(lambda x: x.upper()).value_counts(normalize=True))


P    0.420
R    0.312
I    0.268
Name: Classificação, dtype: float64
P    0.38
R    0.34
I    0.28
Name: Classificação, dtype: float64


In [23]:
#essa parte do codigo salva o resultado do teste em um novo arquivo
x = ""
n = 1
while os.path.isfile('./{0}.xlsx'.format("3080" + x)): #salva um novo arquivo de excel para cada iteracao do loop, com n+1 por iteracao, para comapar as iteracoes do classificador 
    x = "-" + str(n)
    n += 1
file = './{0}.xlsx'.format("3080" + x)
    

writer = pd.ExcelWriter(file) #Abre o arquivo para escrita

test.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False) #salva o arquivo teste classificado


writer.save() #fecha o arquivo

In [27]:
print('teste')
(pd.crosstab(test.Classificação,test.Classificação_A, normalize = 'index')*100).round(5)

teste


Classificação_A,I,P,R
Classificação,,,
I,58.20896,28.35821,13.43284
P,0.95238,59.04762,40.00000
R,0.00000,24.35897,75.64103


In [28]:
print('treinamento')
(pd.crosstab(train.Classificação,train.Classificação_A, normalize = 'index')*100).round(5)

treinamento


Classificação_A,I,P,R
Classificação,,,
I,81.42857,10.00000,8.57143
P,0.00000,94.73684,5.26316
R,0.00000,1.76471,98.23529


In [29]:
print('teste - quantitativo')
pd.crosstab(test.Classificação,test.Classificação_A,)

teste - quantitativo


Classificação_A,I,P,R
Classificação,,,
I,39,19,9
P,1,62,42
R,0,19,59


In [31]:
print('treinamento - quantitativo')
pd.crosstab(train.Classificação,train.Classificação_A)

treinamento - quantitativo


Classificação_A,I,P,R
Classificação,,,
I,114,14,12
P,0,180,10
R,0,3,167


___
### Concluindo

Para concluir e para finalizar o projeto, vamos analisar o nosso sistema de seleção de dados. Em relação ao nosso classificador, nós vemos que comparando a nossa classificacao manual com o classificador, na base de dados de treino, 94% dos tweets pouco relevantes e 98% dos tweets relevantes se manteram entre os dois, porem em torno de 18% dos tweets irrelevantes migraram para outras classificacoes. Enquanto isso na base de dados de teste, obtivemos uma discrepancia maior, nos tweets pouco relevantes quase 60% dos tweets se manteram, mas 40% migraram para relevantes quando aplicamos o classificador, 75% dos tweets relevantes se manteram e 24% migraram para pouco relevantes apos aplicarmos o classificador. Esta discrepancia pode ser epxlicada pelo relaxamento em nosso critero durante a classificacao manual, e pelo fato de ser uma base de dados menor sendo classificada por 3 pessoas, cada uma com uma interpretacao levemente difrente.

Como nossas mensagens são em inglês, a dupla negação seria errado gramaticalmente, não sendo necessária um tratamento para esta questão. Em questão ao tratamento de mensagens sarcásticas, seria muito difícil identificar estas, uma vez que nas redes sociais não há intonação de fala, um dos principais recursos para identificar sarcasmos.

Uma melhoria direta seria ultilizar um algoritmo que usa colocacao e quantidade de emojis nos tweets para identificar sarcasmo, um exemplo e pesquisa direta seria: https://www.media.mit.edu/articles/an-algorithm-trained-on-emoji-knows-when-you-re-being-sarcastic-on-twitter/ 

Nao se pode usar o classificador para gerar mais amostras de treinamento pois estas sao feitas por seres humanos que tem uma interpretacao do que significa ser relevante, irrelevante ou pouco relevante, enquatno o classificador apenas considera a frequencia da palavra no tweet e no banco de dados, assim qualquer variacao na rigorosidade cria um efeito exacerbado no resultado do classificador, ao usalo para gerar novos treinamentos os prorpios erros do classificador serao considerados corretos, e ao longo do tempo ele ira desviar cada vez mais do padrao original, tendendo para um reultado completamente foras do escopo incial.

Em relação ao plano de expansão de nosso projeto, poderíamos tentar ver se conseguiriamos tratar as mensagens sarcásticas, visto que elas podem ter um padrão de palavras como “lol”, “kekw” ou simplesmente um “haha”. Mas visto que estes termos podem estar em mensagens que contém conteúdo não sarcástico, um estudo sobre as mensagens seria importante para evitar este erro, e aumentar a precisão.

Propondo diferentes cenarios para utiliza o classificador Naive-Bayes:

Para melhorar o sistema de blacklist de sites para as redes de escolas, podemos adotar este método para banir sites de acordo com as palavras pesquisadas pelos alunos na internet. 

Um cenário em que uma pessoa é suspeita de um crime, e há poucas provas e evidências, que mostram sua inocência ou culpa. Então, utilizamos o classificador para pegar as palavras mais usadas da pessoa em sites de pesquisa na internet, e passamos elas por um classificador treinado por palavras relevantes ao crime, para ver se a pessoa possui alguma pesquisa suspeita e que crie uma evidência contra ela

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**